In [1]:
# Classification, Intrinsic dimension, Degeneracy
%run SetUp.ipynb

## Configuration 
Title = "Angles"
device = "cuda:1"
pt_option=True

dataset = "MNIST"
width = 2
width_increment = 3
selected_class = 0 ############### selected class #$##########
# selected_class = int(input("Selected class:")) ############### selected class #$##########
dataset_type = "Training set" # "Training + Test set" // "Training set"
data_balace = 1


if dataset == "MNIST":
    image_length = 28
    image_channel=1
    d = 28*28
    n = 60000
    data_balance = 1 ## the lambda value
    Epochs = 30*1000
    start_pruning = 9999 # pretraining

elif dataset == "FashionMNIST":
    image_length = 28
    image_channel=1
    d = 28*28
    n = 60000
    data_balance = 1 ## the lambda value
    Epochs = 30*1000
    start_pruning = 9999 # pretraining

elif dataset == "CIFAR10":
    image_length = 32
    image_channel=3
    d = 3*32*32
    n = 50000
    data_balance = 1 ## the lambda value
    Epochs = 60*1000
    start_pruning = 9999 # pretraining
    
else:
    raise ValueError("Wrong Dataset !")
    

# n = 1000 # for debug, if need

###############################
num_classes = 0 # binary classification
Goal_Accuracy = 99.99 
first_positive_init = True  ## if False, this approximate the class : {selected_class}
############################### if Ture, approximate the complement : {selected_class}^c


# create directory
output_path = f"./Results/{Title}"
create_directory(output_path)
# construct the dataset
trn_X, trn_Y, test_X, test_Y = load_dataset(dataset, n)



if dataset_type == "Training + Test set" : 
    ########### trn + test set #############
    trn_X = torch.cat((trn_X, test_X)) ##### combine test set and trn set !
    trn_Y = torch.cat((trn_Y, test_Y)) ##### 
    ########################################
elif dataset_type == "Training set":
    pass
else:
    raise ValueError(f"Wrong type for the dataset_type : {dataset_type}")

trn_Y = (trn_Y == selected_class).float()# change to binary class ## label 1 for the selected class.
test_Y = (test_Y == selected_class).float()
n = len(trn_X)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|███████████████████████████████████████████████████████████████████| 9912422/9912422 [00:04<00:00, 2393786.57it/s]


Extracting ./DATASETS\MNIST\raw\train-images-idx3-ubyte.gz to ./DATASETS\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 152490.48it/s]


Extracting ./DATASETS\MNIST\raw\train-labels-idx1-ubyte.gz to ./DATASETS\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|████████████████████████████████████████████████████████████████████| 1648877/1648877 [00:02<00:00, 822191.16it/s]


Extracting ./DATASETS\MNIST\raw\t10k-images-idx3-ubyte.gz to ./DATASETS\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|█████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 4676123.90it/s]


Extracting ./DATASETS\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./DATASETS\MNIST\raw



Network Architecture

In [2]:
# Network setting # d -> d1 -> 1
class polytope(nn.Module):
    def __init__(self, width, output_class=1, positive_init=False, small_norm_init=False):
        super(polytope, self).__init__()
        self.fc0 = nn.Linear(d, width)
        self.fc1 = nn.Linear(width, output_class, bias=False)
        self.width = width # width
        self.epoch = 0
        self.data_balance = data_balance
        self.bias = (1 - 2*positive_init)*5 # +-5
        self.positive_init = positive_init
        self.output_class = output_class
        self.small_norm_init = small_norm_init
        self.device = device
        
        # initialization
        if small_norm_init: ## small norm init 은 일반적으로 안좋습니다. !
            self.change_layer_weights(layer=0, W=0.001*torch.randn_like(self.W(0)), b=0.001*(self.b(0))) # multiply 0.01 
            
        if positive_init:
            # if positivie init,  all v_k are positive.
            self.fc1.weight = nn.Parameter(torch.sqrt((self.W(0).norm(dim=1)**2 + self.b(0)**2).view(1,-1)+1))
        else:
            # else, to all v_k are negative.
            self.fc1.weight = nn.Parameter(-torch.sqrt((self.W(0).norm(dim=1)**2 + self.b(0)**2).view(1,-1)+1))
        
        # declare the optimizer
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        
    def forward(self, x):
        self.g1 = self.fc0(x)
        self.h1 = F.relu(self.g1)
        self.g2 = self.fc1(self.h1) + self.bias
        return self.g2.squeeze()
    
    def W(self, layer):
        if layer ==0:
            output = self.fc0.weight
        elif layer ==1:
            output = self.fc1.weight
        return output.clone()
    def b(self, layer):
        if layer ==0:
            output = self.fc0.bias
        elif layer ==1:
            output = self.fc1.bias
        return output if output == None else output.clone()
    
    def activation_pattern(self, x):
        self.forward(x)
        return (self.h1>0).float()
    
    def change_layer_weights(self, layer, W, b):
        if self.b(layer) == None : # no bias term
            if W.shape == self.W(layer).shape and self.b(layer) == b:
                if layer ==0:
                    self.fc0.weight = nn.Parameter(W.to(self.device))
                elif layer ==1:
                    self.fc1.weight = nn.Parameter(W.to(self.device))
            else:
                raise ValueError("wrong shape of input tensors")
        else:
            if W.shape == self.W(layer).shape and b.shape == self.b(layer).shape:
                if layer ==0:
                    self.fc0.weight = nn.Parameter(W.to(self.device))
                    self.fc0.bias = nn.Parameter(b.to(self.device))
                elif layer ==1:
                    self.fc1.weight = nn.Parameter(W.to(self.device))
    #                 self.fc1.bias = nn.Parameter(b)
            else:
                raise ValueError("wrong shape of input tensors")
    
    
    # Training
    def train(self, repetition, trn_X_index):
        self.fail = False
        self.trn_loss_tr = np.empty(0)
        self.test_loss_tr = np.empty(0)
        self.trn_acc_tr = np.empty(0)
        self.test_acc_tr = np.empty(0)

        class0_outPoly_min = 1
        test_acc = 0
        test_loss = torch.zeros(1)
        print(f"=================================   Start Training. Repetition: {repetition+1}   ==================================") if pt_option else None
        time.sleep(1)

        log_period = 500
        for epoch in tqdm(range(Epochs)) :
            self.epoch = epoch
            
            # since class may be imbalance
            loss =  criterion(net(trn_X[trn_X_index][(trn_Y[trn_X_index]==float(positive_init)).view(-1)]).squeeze(),
                              trn_Y[trn_X_index][(trn_Y[trn_X_index]==float(positive_init)).view(-1)]) / len((trn_Y[trn_X_index]==float(positive_init)).view(-1))
            loss += criterion(net(trn_X[trn_X_index][(trn_Y[trn_X_index]==float(not positive_init)).view(-1)]).squeeze(), 
                              trn_Y[trn_X_index][(trn_Y[trn_X_index]==float(not positive_init)).view(-1)]) / len((trn_Y[trn_X_index]==float(not positive_init)).view(-1)) * self.data_balance
            loss *= len((trn_Y[trn_X_index]).view(-1))
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
#             self.pruning(trn_X[trn_X_index])


            if epoch ==0 or epoch % log_period == (log_period-1) :
                self.trn_loss_tr = np.append(self.trn_loss_tr, loss.item())
                self.test_loss_tr = np.append(self.test_loss_tr, test_loss.item())

                trn_acc = ((net(trn_X[trn_X_index])>0).float() == trn_Y[trn_X_index]).sum().item() / len(trn_X[trn_X_index]) *100
                self.trn_acc_tr = np.append(self.trn_acc_tr, trn_acc)
                self.test_acc_tr = np.append(self.test_acc_tr, test_acc)
                class0_inPoly = (self.forward(trn_X[trn_X_index][(trn_Y[trn_X_index]!=float(self.positive_init)).view(-1)]) == self.bias).sum().item()
                class1_inPoly = (self.forward(trn_X[trn_X_index][(trn_Y[trn_X_index]==float(self.positive_init)).view(-1)]) == self.bias).sum().item()
                class0_outPoly = (self.forward(trn_X[trn_X_index][(trn_Y[trn_X_index]!=float(self.positive_init)).view(-1)]) != self.bias).sum().item()
                class1_outPoly = (self.forward(trn_X[trn_X_index][(trn_Y[trn_X_index]==float(self.positive_init)).view(-1)]) != self.bias).sum().item()
                writer.add_scalars("ReLU/Loss", {
                    "Trn_Loss": loss.item(),
                    "Test_Loss": test_loss.item(), }
                                   , epoch+1)
                writer.add_scalars("ReLU/Accuracy", {
                    "Trn_acc": trn_acc,
                    "Test_acc": test_acc, }
                                   , epoch+1)
                writer.add_scalars("ReLU/InOutClass", {
                    "class0_inPoly": class0_inPoly,
                    "class0_outPoly": class0_outPoly,
                    "class0_inPoly": class1_inPoly,
                    "class0_outPoly": class1_outPoly,}
                                   , epoch+1)
                if pt_option :
                    print(f"Epoch: {epoch+1 :>5} | width: {self.width} | loss: {loss.item() :.4f} | ACC: {trn_acc:.3f}", end="")
                    print(f" | Wmin: {self.fc1.weight.abs().min():.1f} | class0_inPoly : {class0_inPoly}", end="")
                    print(f" | class0_outPoly : {class0_outPoly} | class1_inPoly : {class1_inPoly} | class1_outPoly : {class1_outPoly}")
                
                if class0_outPoly == 0: # a good network. save.
                    class0_outPoly_min = 0
                    torch.save(self.state_dict(), folder_name+f"/Rep{repetition+1}_saved_net_width_{width}_min.pt") # save a good polytope
                torch.save(self.state_dict(), folder_name+f"/Rep{repetition+1}_saved_net_width_{width}.pt") # save a good polytope
                
                if self.fc1.weight.sign().sum().abs() !=  self.width: # there is a flipped sign.
                    print("!!! Sign flipped !!") if pt_option else None
                    self.fail = True # skip, train a new network for this round
                    break
                
                if self.width == 0 : # there is no active neuron.
                    print("!!! All neruons have been removed !!") if pt_option else None
                    self.fail = True # skip, train a new network for this round
                    break

                # terminating condition - if one class is completely surrounded by a convex polytope.
                if (class0_outPoly == 0) and (class1_inPoly == 0):
                    print("Found a complete convex polytope cover, escape the training loop") if pt_option else None
                    self.fail = False
                    break
                
                TRN_ACC = (n - class0_outPoly - class1_inPoly) / n * 100
                if (n - class0_outPoly - class1_inPoly) / n * 100 >= Goal_Accuracy :
                    print("Found a 99.99% convex polytope cover, escape the training loop") if pt_option else None
                    self.fail = False
                    break
                
                if ((epoch > 19000) and (epoch % 5000 == 4999)):
                    _a = input("Keep Training? (y/n)")
                    while _a != 'y' and _a != 'n': 
                        print("Wrong input. Please enter again, only (y/n).")
                        _a = input("Keep Training? (y/n)")
                    if _a == 'n':
                        break
                        
                
                # Pruning and merging
                self.pruning(trn_X[trn_X_index]) if epoch > start_pruning else None ## pruning after pre-training
        
        if class0_outPoly_min ==0:
            self.load_state_dict(torch.load(folder_name+f"/Rep{repetition+1}_saved_net_width_{width}_min.pt")) # load the good polytope
        
        # if it fails to achieve convergence : 
        if (class0_outPoly > 10):
            print("!!! It is failed to find a perfect polytope !!!!") if pt_option else None
            self.fail = True # skip, train a new network for this round
#         else: # load the saved network with a good polytope
#             self.load_state_dict(torch.load(folder_name+f"/Rep{repetition+1}_saved_net_width_{width}.pt")) # save a good polytope
# #             self.eval()
                                 
        self.save_loss_graph()
        self.save_result_txt()
        self.save_visualization_weights()
        time.sleep(1)
        print(f"===================================   Training finished, Repetition: {repetition+1}  ================================== \n\n") if pt_option else None
        return TRN_ACC
        
    
    def save_visualization_weights(self):
        # ### visualize and save the image of weight vectors ####
        for _i in range(self.width):
            if dataset != "CIFAR10":
                plt.imshow(self.W(0)[_i].view(image_length,image_length).detach().cpu(), cmap='gray') # black and white image
            else:
                plt.imshow(self.W(0)[_i].view(image_channel,image_length,image_length).permute(1,2,0).clamp(0,1).detach().cpu()) # color image
            plt.title(f"{_i}-th weight")
            plt.savefig(folder_name+f"/w{_i}.png")
            plt.close()
    
    def save_loss_graph(self):
        # plot and save the graphs
        # PLOT THE LOSS GRAPH
        plt.plot(self.trn_loss_tr, label="Train")
        plt.plot(self.test_loss_tr, label="Test")
        plt.xlabel("Iterations (k)")
        plt.ylabel("Loss")
        # plt.yscale("log")
        plt.legend()
        plt.title(f"Rep{repetition+1}_Loss")
        plt.savefig(folder_name+"/loss_fig_fullview.png")
#         plt.show() if pt_option else None
        plt.close()

        # PLOT THE ACCURACY GRAPH
        plt.plot(self.trn_acc_tr, label="Train")
        plt.plot(self.test_acc_tr, label="Test")
        plt.xlabel("Iterations (k)")
        plt.ylabel("Accuracy")
        plt.legend()
        plt.title(f"Rep{repetition+1}_Accuracy")
        plt.savefig(folder_name+"/Accuracy.png")
#         plt.show() if pt_option else None
        plt.close()
    
    # Save result txt file.
    def save_result_txt(self):
        class0_inPoly = (self.forward(trn_X[trn_X_index][(trn_Y[trn_X_index]!=float(self.positive_init)).view(-1)]) == self.bias).sum().item()
        class1_inPoly = (self.forward(trn_X[trn_X_index][(trn_Y[trn_X_index]==float(self.positive_init)).view(-1)]) == self.bias).sum().item()
        class0_outPoly = (self.forward(trn_X[trn_X_index][(trn_Y[trn_X_index]!=float(self.positive_init)).view(-1)]) != self.bias).sum().item()
        class1_outPoly = (self.forward(trn_X[trn_X_index][(trn_Y[trn_X_index]==float(self.positive_init)).view(-1)]) != self.bias).sum().item()
        
        f = open(f"./{folder_name}/Rep{repetition+1}_result.txt", 'w')
        f.write(
        f"""
            This is the CFG file.

            # Accuracy
            Trn_acc : {self.trn_acc_tr[-1] :.3f}
            Test_acc : {self.test_acc_tr[-1] :.3f}

            # Loss
            Trn_loss : {self.trn_loss_tr[-1].item() :.4f} 
            Test_loss : {self.test_loss_tr[-1].item() :.4f} 


            # dataset
            dataset = {dataset}
            n = {n} # number of total data
            d = {d} # dimension
            num_class = {num_classes} # the class, used in this binary classification
            starting_width = {width}
            final_width = {self.width}
            # Network
            {net}
            
            # Polytope
            number of data used in this training (Class 0 / 1): {class0_inPoly+class0_outPoly} / {class1_inPoly+class1_outPoly}
            Class 0 in Polytope : {class0_inPoly}
            Class 1 in Polytope : {class1_inPoly}
            Class 0 out Polytope : {class0_outPoly}
            Class 1 out Polytope : {class1_outPoly}
            
            # optimization
            Epochs = {Epochs}
            lr = {lr}
            optimizer = {self.optimizer}
            Last epoch = {self.epoch+1}
            
            ## trn_X_index
            {trn_X_index}
        """
        )
        f.close()
    
    def pruning(self, dataset):
        width_before = self.width
        dataset_activation_pattern = self.activation_pattern(dataset).sum(dim=0)
        if dataset_activation_pattern.max() == len(dataset) or dataset_activation_pattern.min() == 0 : 
            W0 = self.W(layer=0).detach()
            b0 = self.b(layer=0).detach()
            W1 = self.W(layer=1).detach()
            activation_filter = ((self.activation_pattern(dataset).sum(dim=0) < len(dataset)).float()
                                 * (self.activation_pattern(dataset).sum(dim=0) > 0).float()
                                 * (self.W(layer=1).abs()>0.1).float().view(-1)
                                )
            # change polytope width 
            self.width = activation_filter.sum().long().item()
            width_after = self.width
            self.fc0 = nn.Linear(d, self.width)
            self.fc1 = nn.Linear(self.width, self.output_class, bias=False)
            self.change_layer_weights(layer=0, W=W0[(activation_filter == 1)], b=b0[activation_filter == 1])
            self.change_layer_weights(layer=1, W=W1.view(-1)[activation_filter == 1].view(1,-1), b=None)
            text = "(removing)"
            print(f"\t{text:<15}The network is prunned, width : {width_before} --> {width_after}") if pt_option else None
            
        ## merging vectors with same activation patterns ##
        width_activation_pattern = self.activation_pattern(dataset) # len(dataset) x width
        width_activation_pattern_unique = width_activation_pattern.unique(dim=1).t()
        width_before = self.width
        if len(width_activation_pattern_unique) < self.width :
            self.width = len(width_activation_pattern_unique) 
            width_after = self.width
            W0 = self.W(layer=0).detach()
            b0 = self.b(layer=0).detach()
            W1 = self.W(layer=1).detach()
            self.fc0 = nn.Linear(d, self.width)
            self.fc1 = nn.Linear(self.width, self.output_class, bias=False)
            self.to(device)
            # build weight matrices
            new_W0 = self.W(layer=0).detach()
            new_b0 = self.b(layer=0).detach()
            new_W1 = self.W(layer=1).detach()
            for index, pattern in enumerate(width_activation_pattern_unique):
                new_W0[index] = W0[(width_activation_pattern.t() == pattern).prod(dim=1)==1].sum(dim=0)
                new_b0[index] = b0[(width_activation_pattern.t() == pattern).prod(dim=1)==1].sum(dim=0)
                new_W1[0][index] = W1[0][(width_activation_pattern.t() == pattern).prod(dim=1)==1].sum(dim=0)
            self.change_layer_weights(layer=0, W=new_W0, b=new_b0)
            self.change_layer_weights(layer=1, W=new_W1, b=None)
            text = "(merging)"
            print(f"\t{text:<15}The network is merged, width : {width_before} --> {width_after}") if pt_option else None

        ###### original #######
        smallest_norm_neuron_index = self.width # not index yet.
        smallest_norm = np.inf
        width_activation_pattern = self.activation_pattern(dataset) # len(dataset) x width
        for neuron_index in range(self.width): # for loop
            activated_index = (width_activation_pattern[:, neuron_index] == 1)
            # index 가 activated 된 data들의 activation pattern ## 이게 2 이상이면 제거해버리면 됨 !
            if self.activation_pattern(dataset[activated_index]).sum(dim=1).min().item() >=2 :
                # this neuron can be removed.
                this_round_norm = (self.W(layer=0)[neuron_index].norm() * self.W(layer=1).squeeze()[neuron_index]).item() ## v||w||
#                 this_round_norm = self.W(layer=1).squeeze()[neuron_index].item() ## v
                if this_round_norm < smallest_norm:
                    smallest_norm_neuron_index = neuron_index
                    smallest_norm = this_round_norm
        if smallest_norm_neuron_index < self.width:
            # remove the small-norm redundant neuron, remove only one neuron at once
            width_before = self.width
            width_after = self.width - 1
            W0 = self.W(layer=0).detach()
            b0 = self.b(layer=0).detach()
            W1 = self.W(layer=1).detach()
            self.fc0 = nn.Linear(d, width_after)
            self.fc1 = nn.Linear(width_after, self.output_class, bias=False)
            self.to(device)        
            new_W0 = torch.cat((W0[:smallest_norm_neuron_index], W0[smallest_norm_neuron_index+1:]), dim=0)
            new_b0 = torch.cat((b0[:smallest_norm_neuron_index], b0[smallest_norm_neuron_index+1:]), dim=0)
            new_W1 = torch.cat((W1[:,:smallest_norm_neuron_index], W1[:,smallest_norm_neuron_index+1:]), dim=1)
            self.change_layer_weights(layer=0, W=new_W0, b=new_b0)
            self.change_layer_weights(layer=1, W=new_W1, b=None)
            self.width = width_after
            text = "(pruning)"
            print(f"\t{text:<15}In the polytope, a redundant neuron has been removed") if pt_option else None
        
        
        ### rescaling function ## here, we don't need to rescale the neuron.
        rescale=1.1
        intermediate_index = ( ( F.relu(self.forward(dataset)) != 0).float() * (self.forward(dataset) != self.bias).float() >0).squeeze()
        if intermediate_index.float().sum() >0 :
            activation_pattern_of_intermediate_data = self.activation_pattern(dataset[intermediate_index])
            if activation_pattern_of_intermediate_data.max().item() >0 : ## max
                new_W0 = self.W(layer=0).detach()
                new_b0 = self.b(layer=0).detach()
                new_W1 = self.W(layer=1).detach()
                true_or_false = activation_pattern_of_intermediate_data.sum(dim=0)>0
                for i, boolean in enumerate(true_or_false):
                    if boolean:
                        new_W0[i] = new_W0[i] * rescale
                        new_b0[i] = new_b0[i] * rescale
                        new_W1[0][i] = new_W1[0][i] * rescale
                self.change_layer_weights(layer=0, W=new_W0, b=new_b0)
                self.change_layer_weights(layer=1, W=new_W1, b=None)
                text = "(rescaling)"
                print(f"\t{text:<15}In the polytope, {true_or_false.sum().item()} neurons are rescaled by {rescale}") if pt_option else None
        
        self.optimizer = optim.Adam(self.parameters(), lr=lr) # net. paramgeter 를 optimizer 가능하도록 넣어줘야해...!!

In [3]:
# Optimization setting
lr = 0.0001
criterion = nn.BCEWithLogitsLoss()

# finding the polytope cover !
repetition = 0
trn_X_index_all = (torch.ones(len(trn_X))>0).to(device) # Boolean tensor, shape = len(dataset)
trn_X_index = trn_X_index_all.clone()
network_list = []
folder_name_above = output_path + f'/runs/{dataset}/' + datetime.datetime.now().strftime("%B%d_%H_%M_%S")
create_directory(folder_name_above) # create output folder


complement = "^c" if first_positive_init else ""
print(f"We are finding polytope-basis cover of class [{selected_class}]"+complement+f" of {dataset}, n={n}")
positive_init = first_positive_init
# while trn_X_index[(trn_Y==0).view(-1)].sum()>0 and trn_X_index[(trn_Y==1).view(-1)].sum()>0 : # 100% accuracy
# while trn_X_index[(trn_Y==0).view(-1)].sum()>0 and trn_X_index[(trn_Y==1).view(-1)].sum()>0 : # 원래 쓰던거
trn_acc = 0
while trn_acc < Goal_Accuracy : ## only for single polytope cover !!
    width = int(input("### Please enter the network width: ###")) ## manually adjust the width
    net = polytope(width=width, positive_init=positive_init, small_norm_init=True).to(device)
    folder_name = folder_name_above + f"/Cover_{repetition+1}"
    writer = SummaryWriter(folder_name)
    
    # training
    trn_X_index_this_time = trn_X_index.clone()
    ####
    trn_X_index_this_time[(trn_Y==float(positive_init)).view(-1)] = True  #### consider all data points in the other class
    ####
    trn_acc = net.train(repetition=repetition, trn_X_index= trn_X_index_this_time)
    again = input("### Do you wanna train again? (y/n) ###")
    while again != 'y' and again != 'n': 
        print("Wrong input. Please enter again, only (y/n).")
        again = input("### Do you wanna train again? (y/n) ###")
    if again == 'y':
        net.fail = True
    if net.fail:
        # there is some problem for finding a cover in this training time.
        # therefore, we skip this try and re-train.
#         positive_init = not positive_init # flip the convexity of the class.
        continue ## CAUTION : this might occur infinite loop
    
    # otherwise, it was successful.
    # after trainnig, extract the index.
    trn_X_index_in_polytope = ((net(trn_X).view(-1) * trn_X_index)  == net.bias) # boolean tensor, shape = len(dataset)
    # new index
    trn_X_index = trn_X_index * trn_X_index_in_polytope
        
    network_list.append(net) # add to the network_list
    positive_init = not positive_init # flip the convexity of the class.
    repetition += 1

# result
if pt_option :
    print("Completed to find a polytope-basis cover.")
    print(f"There are {len(network_list)} polytopes.") 
    print(network_list)

We are finding polytope-basis cover of class [0]^c of MNIST, n=60000


### Please enter the network width: ### 10


=================================   Start Training. Repetition: 1   ==================================


  0%|                                                                              | 1/30000 [00:00<4:03:34,  2.05it/s]

Epoch:     1 | width: 10 | loss: 4.8870 | ACC: 90.128 | Wmin: 1.0 | class0_inPoly : 0 | class0_outPoly : 54077 | class1_inPoly : 0 | class1_outPoly : 5923


...

	(rescaling)    In the polytope, 10 neurons are rescaled by 1.1


 42%|███████████████████████████████▏                                           | 12491/30000 [01:42<02:21, 124.08it/s]

Epoch: 12500 | width: 10 | loss: 0.0072 | ACC: 100.000 | Wmin: 2.3 | class0_inPoly : 52670 | class0_outPoly : 1407 | class1_inPoly : 0 | class1_outPoly : 5923


 42%|███████████████████████████████▏                                           | 12499/30000 [01:43<02:24, 120.91it/s]


KeyboardInterrupt: 

In [ ]:
### Result ####
width_list = [net.width for net in network_list]
print(f"Output: a polytope-basis cover of class [{selected_class}]{complement}")
print(f"There are totally {len(network_list)} polytopes. The widths are",width_list,end="\n\n")

#### Accuracy #########
# trn acc
previous_trn_answer = torch.zeros_like(trn_Y)
for j, net in enumerate(network_list):
    previous_trn_answer += (-1)**j * (net(trn_X) == net.bias).float()
correct_trn_data = ((previous_trn_answer>0).float() == float(first_positive_init) + (1-2*(float(first_positive_init)))*trn_Y).sum().item()
trn_acc =  correct_trn_data / len(trn_Y) * 100
print(f"Trn ACC : {trn_acc:.2f} %  ||  # incorrect data : {len(trn_Y) - correct_trn_data} / {len(trn_Y)}")

# test ACC
previous_test_answer = torch.zeros_like(test_Y)
for j, net in enumerate(network_list):
    previous_test_answer += (-1)**j * (net(test_X) == net.bias).float()
correct_test_data = ((previous_test_answer>0).float() == float(first_positive_init) + (1-2*(float(first_positive_init)))*test_Y).sum().item()
test_acc =  correct_test_data / len(test_Y) * 100
print(f"Test ACC : {test_acc:.2f} %  ||  # incorrect data : {len(test_Y) - correct_test_data} / {len(test_Y)}")

f = open(f"./{folder_name_above}/Total_result.txt", 'w')
f.write(
f"""
    This is the result file
    
    # dataset
    dataset = {dataset}, {dataset_type}
    n = {len(trn_X)} # number of trained data
    d = {d} # dimension
    Initialization width = {width}
    
    # class, convexity
    selected_class = {selected_class} # the class, used in this binary classification
    positive_init = {first_positive_init}
    Therefore, it finds a polytope-basis cover of class [{selected_class}]{complement})
    data_balance = {data_balance}
    
    # Accuracy
    Trn ACC : {trn_acc:.2f} %  ||  # incorrect data : {len(trn_Y) - correct_trn_data} / {len(trn_Y)}
    Test ACC : {test_acc:.2f} %  ||  # incorrect data : {len(test_Y) - correct_test_data} / {len(test_Y)}
    
    # optimization
    Epochs = {Epochs}
    Start_pruning = {start_pruning}
    
    There are totally {len(network_list)} polytopes. The widths are
    {width_list}
    
    The end.
"""
)
f.close()

In [ ]:
# ACC
previous_trn_answer = torch.zeros_like(trn_Y)
for j, net in enumerate(network_list):
    previous_trn_answer += (-1)**j * (net(trn_X) == net.bias).float()
correct_trn_data = ((previous_trn_answer>0).float() == float(first_positive_init) + (1-2*(float(first_positive_init)))*trn_Y).sum().item()
trn_acc =  correct_trn_data / len(trn_Y) * 100
print(f"Trn ACC : {trn_acc:.2f} %  ||  # incorrect data : {len(trn_Y) - correct_trn_data} / {len(trn_Y)}")

### Row-interchange, Column-interchange, useful functions

In [ ]:
# imshow(A<.9)
# imshow(A)

#### change row ####
def change_row(A, i, j):
    A[[i, j]] = A[[j, i]]
    return A
### change column ####
def change_column(A, i, j):
    B = A.t()
    B[[i, j]] = B[[j, i]]
    return B.t()

### rearrangement to be diagonal
def rearrangement(A):
    assert A.dim() == 2
    assert A.size(0) == A.size(1)
    B = A.clone()
    m = B.size(0)
    horizontal_index = torch.arange(m).view(-1,1)
    vertical_index = torch.arange(m).view(1,-1)
    
    for i in range(m):
        # i 번째 row 에서 정렬.
        j = B[i].argmin().item()
        B = change_column(B, i, j) ;     vertical_index = change_column(vertical_index, i, j)
        
    # rearrange, w.r.t. diagonal entries
    C = B.clone()
    stop = False
    while not stop:
        for i,j in enumerate(B.diag().topk(m, largest=False)[1]):
            j = j.item()
            if i != j :
                B = change_column(B,i,j) ;    vertical_index = change_column(vertical_index, i, j)
                B = change_row(B,i,j) ;       horizontal_index = change_row(horizontal_index, i, j)
                break
            if i == j and j == (m-1):
                stop = True
    return B, horizontal_index, vertical_index

### Load two saved polytope covers.

In [ ]:
## analysis - unique solution ?
selected_class = 0
width = 4
net1 = polytope(width=width, positive_init=positive_init).to(device)
net1.load_state_dict(torch.load(f"./Results/Rebuttal/runs/{dataset}_saved/m_{selected_class}_{width}/Cover_1/Rep1_saved_net_width_{width}.pt"))
W1 = net1.W(0)

net2 = polytope(width=width, positive_init=positive_init).to(device)
net2.load_state_dict(torch.load(f"./Results/Rebuttal/runs/{dataset}_saved/m_{selected_class}_{width}/Cover_1/Rep1_saved_net_width_{width}.pt"))
W2 = net2.W(0)


A = torch.zeros(width, width)
for a in range(width):
    for b in range(width):
        A[a][b] = (protractor(W1[a],W2[b]).item())
# print(A)
B, horizontal_index, vertical_index = rearrangement(A) ### rearrangement 
# print("Arranged")
W1 = W1[horizontal_index.view(-1)] ### rearrangement 
W2 = W2[vertical_index.view(-1)] ### rearrangement 
for a in range(width):
    for b in range(width):
        A[a][b] = (protractor(W1[a],W2[b]).item())
fig, axs = plt.subplots(2,width, figsize=(2*width*3,width*3))
for i in range(width):
    axs[0][i].imshow(W1[i].view(28,28).detach().cpu(), cmap='gray')
#     axs[0][i].set_title(f"w{i+1}")
    axs[1][i].imshow(W2[i].view(28,28).detach().cpu(), cmap='gray')
    axs[0][i].axis("off")
    axs[1][i].axis("off")
plt.savefig(folder_name_above+f"/image_class{selected_class}.png")
plt.show()

print(A)
fig = plt.figure(figsize = (2*width,2*width))
ax = fig.add_subplot(111)
ax.imshow(A, cmap='gray')
xticks_label = []
yticks_label = []
for i in range(width):
    xticks_label.append(r"$\tilde{w}$"+fr"$_{i+1}$")
    yticks_label.append(fr"$w_{i+1}$")
    for j in range(width):
        ax.text(i,j,f"{A[j][i].item(): .4f}", color='red', ha='center', va='center', fontsize=20)
fig.suptitle(r"Angle bewteen $w_i$ and $\tilde{w}_j$", fontsize=20)
plt.xticks(np.arange(width))
plt.yticks(np.arange(width))
ax.set_xticklabels(xticks_label, fontsize=20)
ax.set_yticklabels(yticks_label, fontsize=20)
plt.savefig(folder_name_above+f"/Angle_color_class{selected_class}.png")
plt.show()